In [2]:
import googlemaps
import os
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List, Literal, Optional
from tavily import TavilyClient
from pydantic import BaseModel, Field
from serpapi import GoogleSearch

In [ ]:
os.environ["GPLACES_API_KEY"] = ""
os.environ["GOOGLE_API_KEY"] = ""
os.environ["TAVILY_API_KEY"] = ""
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"
tavily_api = os.environ["TAVILY_API_KEY"]
api =""
gmaps = googlemaps.Client(api)
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, max_retries = 3)
tavily_client = TavilyClient(tavily_api)

In [ ]:
# Initializing the Base Class to ensure correct formatting of model
class DistanceToolFormatting(BaseModel):
    origins: str = Field(description = "The starting point from where we want to see")
    destinations: str = Field(description = "The Ending Point which is used to calculate the distance")
    mode: Literal["walking", "driving"] = Field(default = "walking", description = "The mode which is used to calculate the duration")

In [ ]:
def flight_info_extract(flight_option, departure_id: str, arrival_id: str, travel_date: str):
    all_options= []
    # other_flights = results['other_flights']
    if not flight_option:
        result = f"There are no Flights from: {departure_id} to {arrival_id} on date: {travel_date}"
        return all_options, result
    for idx, flight in enumerate(flight_option):
        price = flight.get("price", -1)
        if price >= 0:
            all_options.append(
                {
                    "Flight Option": idx+1,
                    "price": price,
                    "Layover": [layover_name.get('name', "") for layover_name in flight.get('layovers', [])] if flight.get('layovers') else ["No Layover"],
                    "Total Flight Duration (hours)": round(flight.get('total_duration', 0)/60, 1)
                }
            )
        else:
            continue
    all_options.sort(key = lambda x: x['price'])
    if not all_options:
        result = f"There is no Price available for all flights."
        return all_options, result 
    result = f"There are Flights from: {departure_id} to {arrival_id} on date: {travel_date}" 
    return all_options, result 


@tool
def One_way_flight_search(departure_id: str, arrival_id: str, travel_date: str):
    """
   Search for one-way flights between two airports on a specific date.
    Returns flight options with price, layovers, and duration details.
    
    Args:
        departure_id (str): Departure airport code (e.g., 'PEK' for Beijing)
        arrival_id (str): Arrival airport code (e.g., 'ISB' for Islamabad)
        travel_date (str): Travel date in YYYY-MM-DD format (e.g., '2026-03-03')
    
    Returns:
        str: Formatted flight search results with prices, durations, and layover information
    
    """
    params = {
    "engine": "google_flights",
    "departure_id": departure_id,
    "arrival_id": arrival_id,
    "currency": "USD",
    "type": "2",
    "outbound_date": travel_date,
    "api_key": ""
    }
    try:
        search = GoogleSearch(params)
        results = search.get_dict()
    except Exception as e:
        return f"Failed to Fetch Flight Details: {e}"

    if results.get('error'):
        return f"There is some issue with the results: {results['error']}"
    lowest_available_price = results['price_insights']['lowest_price']
    current_price_status = results['price_insights']['price_level']
    average_price_status= results['price_insights']['typical_price_range']
    departure_airport = results['airports'][0]['departure'][0]['airport']['name']
    arrival_airport = results['airports'][0]['arrival'][0]['airport']['name']
    best_flight_result, best_flight_remarks = flight_info_extract(results.get("best_flights", []), departure_airport, arrival_airport, travel_date)
    other_flight_result, other_flight_remarks = flight_info_extract(results.get("other_flights", []), departure_airport, arrival_airport, travel_date) 
    final_result = f"""
    Departure Airport {departure_airport}({departure_id})
    Arrival Airport {arrival_airport}({arrival_id})
    Lowest Available Price: {lowest_available_price}
    Current Prices Status: {current_price_status}
    Average Price Status: {average_price_status}
    Best Flight Information:
        There are total of {len(best_flight_result)} flight options.
        Remarks on Best Flight: {best_flight_remarks}
        Overall Best Flight Info: {best_flight_result}
    Other Flight Information:
        There are total of {len(other_flight_result)} flight options.
        Remarks on Other Flights: {other_flight_remarks}
        Overall Other Flight Info: {other_flight_result}
    """
    print(f"Flight search Tool: {final_result}")
    return final_result


In [6]:
# Tool Calling for distance matrix calculation
@tool("distance_matrix", description = "This tool is used to measure the distance between two locations", args_schema = DistanceToolFormatting)
def distance_measurement_tool(origins: str, destinations: str, mode: str = "walking"):
    """ This function is used to measure the distance between two locations. 
        The locations should be in str and the origins is the location person want to go from and destinations is the location person want to go to
        The mode is the mode of travel, it can be "walking" or "driving" 
        The return or outputs of this tools is string which contain the distance and time taken. 
    """
    result = gmaps.distance_matrix(origins, destinations, mode = mode)
    print(f"The result from distance tool: {result}")
    distances = []
    durations = []

    for element in result['rows']:
        for status in element['elements']:
            if status['status'] == "ZERO_RESULTS":
                return (
                    "No ground route found between the selected locations. "
                    "This route may cross restricted borders or require air travel."
                )
    for row in result['rows']:
        for sub_row in row["elements"]:
            distance = sub_row['distance']['text']
            duration = sub_row['duration']['text']
            distances.append(distance)
            durations.append(duration)
    final_formatted_result = f"The duration between {origins} and {destinations} is {durations} min and distance is {distances} km"
    return final_formatted_result

In [7]:
@tool("Web_Search_Tool", description = " This tool is used when there is a need to search online.")
def web_search_tool(
    query: str, 
    country: str = "us", 
    max_results: int = 5,
    search_depth: Literal["basic", "advanced"] = "advanced",
    topic: Literal["general", "news"] = "general"
):
    """
    Perform an advanced web search using the Tavily API to find real-time information, 
    news, or travel itineraries.
    
    Use this tool when you need to find:
    - Current events or news (set topic="news")
    - Travel itineraries, guides, and blogs
    - Facts that are not in your internal knowledge base
    
    Args:
        query (str): The search query. Be specific (e.g., "3 day itinerary for Tokyo" instead of "Tokyo").
        country (str): The 2-letter ISO country code to bias results towards (e.g., "us", "fr", "jp"). Defaults to "us".
        max_results (int): The maximum number of search results to return. Defaults to 5.
        search_depth (str): "basic" for fast results, "advanced" for high-quality content. Defaults to "advanced".
        topic (str): "general" for most queries, "news" for recent events. Defaults to "general".

    Returns:
        list[dict]: A list of dictionaries containing 'url', 'content', and 'title' for each result.
    """
        
    try:
        if topic == "general":
            response = tavily_client.search(
                query=query,
                search_depth=search_depth,
                topic=topic,
                country=country,
                max_results=max_results
            )
        else:
            response = tavily_client.search(
                query=query,
                search_depth=search_depth,
                topic=topic,
                max_results=max_results
            )
        return response.get("results", [])

    except Exception as e:
        return f"Error performing search: {str(e)}"


In [8]:
# System Prompt Defining
system_prompt = """
You are a Travel Planning Assistant specialized in logistics and itinerary feasibility.

TOOLS:
- distance_matrix: Use ONLY to compute driving/walking distance and travel duration for ground transport.
- Web_Search_Tool: Use ONLY for current or externally verifiable information (guides, attractions, news).
- One_way_flight_search: Use ONLY for finding one-way flight prices, layovers, and durations between airports.

RULES:
1. Never estimate or hallucinate distances, durations, or flight prices.
2. Always use distance_matrix when ground travel time or distance is requested.
3. Always use One_way_flight_search for queries involving air travel, flight costs, or layovers. 
   - You MUST obtain a valid travel date (YYYY-MM-DD), departure city, and arrival city before calling this tool. 
   - If the date is missing, ask the user for it.
4. Always use Web_Search_Tool for finding attractions, travel guides, or general current events.
5. Ask for clarification if locations are ambiguous (e.g., "Paris" could be France or Texas).
6. If tools fail, explain the issue and request an alternative location or date.

TOOL SELECTION:
- Ground travel (Km/Min) → distance_matrix
- Facts/News/Attractions/Itineraries → Web_Search_Tool
- Flight Prices/Durations/Layovers → One_way_flight_search

OUTPUT:
- Be concise and practical.
- For flights, explicitly highlight the cheapest option vs. the fastest option.
- Add brief context (e.g., "This is a walkable distance" vs. "This requires a long flight").
- Clearly state any assumptions made.
"""
# create agent
agent = create_agent(model = model, tools = [distance_measurement_tool, web_search_tool, One_way_flight_search], system_prompt = system_prompt)

In [18]:
# Agent Invoking
res = agent.invoke({"messages": "I want to go to Islamabad from China, tell me best option from shanghai, beijing, xi'an, guangzhou on 15th march 2026"})

Flight search Tool: 
    Departure Airport Xi'an Xianyang International Airport(XIY)
    Arrival Airport Islamabad International Airport(ISB)
    Lowest Available Price: 385
    Current Prices Status: low
    Average Price Status: [400, 420]
    Best Flight Information:
        There are total of 3 flight options.
        Remarks on Best Flight: There are Flights from: Xi'an Xianyang International Airport to Islamabad International Airport on date: 2026-03-15
        Overall Best Flight Info: [{'Flight Option': 1, 'price': 385, 'Layover': ['Beijing Daxing International Airport', 'Ürümqi Diwopu International Airport'], 'Total Flight Duration (hours)': 20.5}, {'Flight Option': 2, 'price': 385, 'Layover': ['Beijing Daxing International Airport', 'Ürümqi Diwopu International Airport'], 'Total Flight Duration (hours)': 20.5}, {'Flight Option': 3, 'price': 633, 'Layover': ['Guangzhou Baiyun International Airport'], 'Total Flight Duration (hours)': 12.1}]
    Other Flight Information:
       

In [19]:
print(res['messages'][-1].content[0]["text"])

Here are the flight options from Shanghai, Beijing, Xi'an, and Guangzhou to Islamabad on March 15, 2026:

**From Shanghai (PVG):**
*   No flights were found for this date.

**From Beijing (PEK):**
*   **Cheapest Option:** 443 with one layover at Suvarnabhumi Airport, total duration of 32.2 hours.
*   **Fastest Option:** 2418 with two layovers at Shanghai Pudong International Airport and Zayed International Airport, total duration of 23.2 hours.

**From Xi'an (XIY):**
*   **Cheapest Option:** 385 with two layovers at Beijing Daxing International Airport and Ürümqi Diwopu International Airport, total duration of 20.5 hours.
*   **Fastest Option:** 633 with one layover at Guangzhou Baiyun International Airport, total duration of 12.1 hours.

**From Guangzhou (CAN):**
*   **Cheapest Option:** 443 with one layover at Ürümqi Diwopu International Airport, total duration of 15.9 hours.
*   **Fastest Option:** 520 with no layovers, total duration of 6.9 hours.

**Summary of Best Options:**

*  